In [1]:
using LinearAlgebra, PyPlot

In [2]:
function measurements(a::Array{Float64, 1}, x::Array{Float64, 1}, fc::Int64)
    y = zeros(Complex{Float64}, fc + 1);
    y[1] = sum(a);
    for k in 2:fc + 1
        y[k] = dot(a, exp.(-im*(k - 1)*x));
    end
    return y
end

function toeplitzMeasurements(y::Array{Complex{Float64}, 1})
    fc = length(y) - 1;
    T = zeros(Complex{Float64}, fc + 1, fc + 1)
    for i in 1:fc
        for j in i + 1:fc + 1
            a = y[j - i + 1];
            T[i, j] = a;
            T[j, i] = conj(a);
        end
    end
    T = T + diagm(0 => y[1].*ones(fc + 1));
    return T;
end

function vandermonde(x::Array{Float64, 1}, fc::Int64)
    K = length(x);
    V = zeros(Complex{Float64}, fc + 1, K);
    for k in 1:K
        V[:, k] = collect(exp.(im*(0:fc)*x[k]));
    end
    return V
end

vandermonde (generic function with 1 method)

In [3]:
function toeplitzCleaning(T::Array{Complex{Float64},2})
    # T is supposed to be Hermitian and p.s.d
    # This function makes sure of it
    if !ishermitian(T)
        T = Hermitian(T);
    end
    lambda, P = eigen(T);
    for i in 1:length(lambda)
        if lambda[i] < 1e-15
            lambda[i] = 1e-15; # margin w.r.t 0. If 0.0 or eps(Float64) then with the rounding errors,
            # might end up with a non p.s.d matrix
        end
    end
    P_tilde = P*sqrt.(diagm(0 => lambda));
    T_new = P_tilde*P_tilde'; # since of the form A*A' => preserve Hermitian. Remains p.s.d with the margin
    if norm(T_new - T) > 1e-13
        println(norm(T_new - T))
    end
    return T_new;
end

toeplitzCleaning (generic function with 1 method)

In [4]:
function CFP(T::Array{Complex{Float64},2})
    # Algorithm from yang2018frequency to get the CFP decomposition
    # see just after Theorem 1
    
    # does not seem to be very stable :
    # - isssue with positivity of T => issue in Cholesky decomposition - FIXED
    # - computations of weights not very stable - FIXED
    # - positions are fine.
    
    r = rank(T);
    y = T[1, :];
    fc = length(y) - 1;
    concat = false;
    if r == size(T)[1] # if T is full rank (hence p.d.) then we substract to T, a rank one matrix
        x0 = rand()*2*pi;
        e = conj.(measurements([1.0], [x0], fc));
        e_t = T\e;
        a0 = 1/real(e'*e_t);
        T = T - a0*e*e';
        r = r - 1;
        concat = true;
    end
    
    T = toeplitzCleaning(T); # make T Hermitian and psd if not the case    
    if ishermitian(T) && isposdef(T) # we still make the sanity check, just in case
        a, x = zeros(r), zeros(r);
        C = cholesky(T, check = true); # T = C.L*C.L'
        V = C.L[:, 1:r]; # the last columns of C.L are zero
        v1 = V[1, :];
        Vu = V[1:end-1, :];
        Vl = V[2:end, :];
        F = schur(Vu'*Vl, Vu'*Vu);
        z = F.α./F.β # generalized eigenvalues : coeff satisfies (A - z*B)*u = 0
        x = imag(log.(z)) # to get the positions
        x = x - floor.(x./(2*pi))*2*pi; # put everyone in torus
            ########### Previous algorithm for weight estimation : not stable ! Simpler to solve directly linear system
            # k = 1;
            # for i in 1:length(z) # to get the generalized eigenvector associated to z. Compute kernel of (A - z*B)
            #     eig, vec = eigen(Vu'*Vl - z[i]*Vu'*Vu);
            #     for j in 1:length(eig)
            #         if norm.(eig)[j] < 1e-10 # only one
            #             u = vec[j, :] # only vector in the kernel of (A - z*B). This is the 
            #             a[k] = norm(v1'*u)^2;
            #             k += 1;
            #         end
            #     end
            # end
            ###########
        if concat
            a, x = vcat([a0], a), vcat([x0], x);
        end
        # much better way to get weights
        V = conj.(vandermonde(x, length(y)-1));
        a = real.(V\y);
        #
        return a, x;
    else
        println("T is hermitian: ", ishermitian(T), ", T is psd: ", isposdef(T))
        return 0, 0;
    end
end

CFP (generic function with 1 method)

In [5]:
# Example 1
# - positive measure
# - K <= fc

In [6]:
K = 7;
a0 = .5*ones(K);
x0 = 2*pi*rand(K);
fc = 10;
y = measurements(a0, x0, fc);
T = toeplitzMeasurements(y);
isposdef(T)

false

In [7]:
println("eigen values: ", eigen(T).values)
println("rank(T): ", rank(T))

eigen values: [-1.16981e-15, 2.03927e-16, 7.58914e-16, 1.2955e-15, 0.802368, 3.17471, 5.08731, 5.74432, 6.52355, 7.24128, 9.92646]
rank(T): 7


In [8]:
# We expect a unique solution composed of rank(T) positive Dirac masses. It is the one who produced y

In [9]:
a, x = CFP(T); # the Caratheodory - Fejer - Pisarenko decomposition gives in this case the unique sol

In [10]:
a, sort(x)

([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.606357, 2.24745, 2.67803, 4.08024, 4.31081, 4.78609, 6.27796])

In [11]:
a0, sort(x0)

([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.606357, 2.24745, 2.67803, 4.08024, 4.31081, 4.78609, 6.27796])

In [12]:
# Example 2
# - positive measure
# - K > fc

In [13]:
K = 6;
a0 = .5*ones(K);
x0 = 2*pi*rand(K);
fc = 4;
y = measurements(a0, x0, fc);
T = toeplitzMeasurements(y);
isposdef(T)

true

In [14]:
println("eigen values: ", eigen(T).values)
println("rank(T): ", rank(T))
# T is p.d.

eigen values: [0.0746244, 2.18863, 2.5412, 4.91469, 5.28085]
rank(T): 5


In [15]:
# We expect to have infinitely many solutions composed of fc+1 positive Dirac masses.

In [16]:
a, x = CFP(T);
I = sortperm(x);

In [17]:
a[I], x[I]

([0.507996, 0.141671, 0.839085, 0.951006, 0.560241], [0.318167, 2.72341, 3.1864, 4.33867, 5.31365])

In [18]:
norm(y - measurements(a, x, fc)) # if very small, proves that (a, x) is indeed a solution.

1.8784845624986776e-14

In [19]:
# Example 3
# - measure with positive and negative Dirac masses
# - K > fc

In [20]:
K = 6;
a0 = .5*ones(K);
a0[1] = -1.0
x0 = 2*pi*rand(K);
fc = 8;
y = measurements(a0, x0, fc);
T = toeplitzMeasurements(y);
isposdef(T)

false

In [21]:
println("eigen values: ", eigen(T).values)
println("rank(T): ", rank(T))
# T is p.d.

eigen values: [-8.55779, -3.11358e-15, 4.04332e-16, 4.43165e-15, 0.131803, 3.55042, 4.12926, 4.93262, 9.31368]
rank(T): 6


In [22]:
# case of uniqueness

In [23]:
# test random y

In [24]:
fc = 7;
b = randn(fc) + im*randn(fc)
y = vcat([maximum(abs.(b)) + 1.0], b)

8-element Array{Complex{Float64},1}:
   2.8131273910898997 + 0.0im                
  0.40723520419820314 + 0.5248538162483616im 
    1.266744785823615 - 0.08428699475291893im
  -1.0537261406013065 - 1.4754972575148835im 
    1.402313645862106 - 0.28082749280509944im
  0.41321780290338794 + 1.4092758846247537im 
   0.6996456078936116 - 0.5054219338112677im 
 -0.28710186536172705 + 0.3222172493160041im 

In [25]:
T = toeplitzMeasurements(y);

In [26]:
eigen(T).values

8-element Array{Float64,1}:
 -2.5190842293325106
 -1.4345841886760562
  1.0606645332570057
  2.232715662782808 
  3.8648877848263345
  5.8645108802126895
  6.361891601772338 
  7.074017083876605 